In [1]:
import os
! cd .. && python -m pip install .

Defaulting to user installation because normal site-packages is not writeable
Processing /home/jsteinbauer/ondewo/ondewo-s2t-client-python
  Using cached protobuf-3.19.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.1 MB)
  Created wheel for ondewo-s2t-client: filename=ondewo_s2t_client-1.4.1-py2.py3-none-any.whl size=20204 sha256=c714c1faea03e3a851f0ab02da82ecf9dc2d14d721fd1260bc12cfb70a078863
  Stored in directory: /home/jsteinbauer/.cache/pip/wheels/68/33/5c/6741336a8de21fa9ea1d4ff5e96fc434fb52c5c39721dc69c7
Successfully built ondewo-s2t-client
  Attempting uninstall: ondewo-s2t-client
    Found existing installation: ondewo-s2t-client 1.4.1
    Uninstalling ondewo-s2t-client-1.4.1:
      Successfully uninstalled ondewo-s2t-client-1.4.1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your pack

In [2]:
import argparse
import wave

from ondewo.s2t import speech_to_text_pb2
from ondewo.s2t.client.client import Client
from ondewo.s2t.client.client_config import ClientConfig
from ondewo.s2t.client.services.speech_to_text import Speech2Text
from ondewo.s2t.speech_to_text_pb2 import ListS2tPipelinesRequest, Speech2TextConfig

## Creating a client object
The example below shows how to create a speech to text client object from a client config. 
When setting *use_secure_channel=True*, a grpc certificate *grpc_cert* is required.

In [5]:
AUDIO_FILE: str = "examples/audiofiles/sample_1.wav"
# credentials = grpc.ssl_channel_credentials(root_certificates=cert)

MAX_MESSAGE_LENGTH: int = 60000000
GRPC_HOST: str = "localhost" #"<ADD GRPC SERVER HERE>"
GRPC_PORT: str = "50655" #"<ADD GRPC PORT HERE>"
CHANNEL: str = f"{GRPC_HOST}:{GRPC_PORT}"
grpc_cert: str = None #"<ADD CERTIFICATE HERE>"


options = [
    ('grpc.max_send_message_length', MAX_MESSAGE_LENGTH),
    ('grpc.max_receive_message_length', MAX_MESSAGE_LENGTH),
]

# channel = grpc.secure_channel(CHANNEL, credentials, options=options)

config: ClientConfig = ClientConfig(
  host=GRPC_HOST,
  port=GRPC_PORT, 
  grpc_cert=grpc_cert)
    
print(config)
    
client: Client = Client(config=config, use_secure_channel=False)

ClientConfig(host='localhost', port='50655', grpc_cert=None)


## List all existing speech to text pipelines
All relevant configurations of the speech to text server are defined in a speech to text pipeline. 
A running server can store several such configs at the same time, such that the client can chose which one to 
pick when he sends a request to transcribe an audio file or stream.

The example below shows how to list all available pipelines by calling the **s2t_service.list_s2t_pipelines()** function, which takes a **ListS2tPipelinesRequest()** as an argument.


In [6]:
s2t_service: Speech2Text = client.services.speech_to_text
pipelines = s2t_service.list_s2t_pipelines(request=ListS2tPipelinesRequest()).pipeline_configs
for pipeline in pipelines:
    print(pipeline)

id: "quarznet_en"
description {
  language: "en"
  pipeline_owner: "ONDEWO"
  domain: "general"
}
active: true
inference {
  ctc_acoustic_models {
    type: "quartznet"
    quartznet {
      config_path: "acoustic_models/model_configs/en/quartznet15x5.yaml"
      load_type: "pt_files"
      pt_files {
        path: "acoustic_models/en/quartznet15x5/"
        step: "247400"
      }
      ckpt_file {
        path: "acoustic_models/de/ckpt_test/test.ckpt"
      }
      use_gpu: true
    }
    quartznet_triton {
      config_path: "acoustic_models/model_configs/en/quartznet15x5.yaml"
      triton_url: "localhost:8001"
      triton_model: "quartznet"
    }
    wav2vec {
      model_path: "acoustic_models/en/wav2vec/wav2vec2-base-960h"
      use_gpu: true
    }
  }
  language_models {
    path: "language_models/ngram_lm_models/test/en/"
    beam_size: 64
    default_lm: "en_small"
    beam_search_scorer_alpha: 2.0
    beam_search_scorer_beta: 1.0
  }
}
streaming_server {
  output_style: "sim

In [7]:
import wave 
with wave.open("audiofiles/sample_1.wav") as w:
    audio: bytes = w.readframes(w.getnframes())

In [8]:
from typing import List, Optional

def find_pipeline_for_language(pipelines: List[Speech2TextConfig], language: str) -> Optional[Speech2TextConfig]:
    """ 
    Returns the first speech to text pipeline for the requested language. 
    If no pipline is found, return None.
    """
    for pipeline in pipelines:
        if pipeline.description.language == language:
            return pipeline

english_pipeline = find_pipeline_for_language(pipelines=pipelines, language='en')
german_pipeline = find_pipeline_for_language(pipelines=pipelines, language='de')

## Send a transcribe file request to the server
In general, are two different endpoints for audio transcriptions:
1. **Transcribe an audio file** 
2. **Transcribe an audio stream**

In this example, we create a **TranscribeFileRequest**, including the audio file (as bytes) and a **TranscribeRequestConfig** message, including the speech to text pipeline id, as well as optional additional parameters.
The request message is then used as an argument to the **s2t_service.transcribe_file()**, which calls the corresponding endpoint.

In [9]:
pipeline: Speech2TextConfig = pipelines[1]

request = speech_to_text_pb2.TranscribeFileRequest(
    audio_file=audio,
    config=speech_to_text_pb2.TranscribeRequestConfig(
        s2t_pipeline_id=pipeline.id,
        ctc_decoding=speech_to_text_pb2.CTCDecoding.BEAM_SEARCH_WITH_LM,
    )
)
# Send transcription request and get response
transcribe_response = s2t_service.transcribe_file(request=request)

for transcription_message in transcribe_response.transcriptions:
    print(f"File transcript: {transcription_message.transcription}")

File transcript: der erste der januar siebzehnte sechste 
